### Process Results

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read from Bronze Layer

In [ ]:
results_df = spark.read.table("f1_bronze.results")

##### Step 2 - Rename and Drop Columns

In [ ]:
from pyspark.sql.functions import col

results_renamed_df = (
    results_df.withColumnRenamed("resultId", "result_id")
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("positionText", "position_text")
    .withColumnRenamed("positionOrder", "position_order")
    .withColumnRenamed("fastestLap", "fastest_lap")
    .withColumnRenamed("fastestLapTime", "fastest_lap_time")
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")
    .drop("statusId")
)

##### Step 3 - Deduplicate

In [ ]:
results_deduped_df = results_renamed_df.dropDuplicates(["race_id", "driver_id"])

##### Step 4 - Add Ingestion Date

In [ ]:
results_final_df = add_ingestion_date(results_deduped_df)

##### Step 5 - Write to Silver Layer
Using generic merge capability or partition overwrite.
For simplicity and robustness with the provided Utils pattern (which I refactored slightly), I will use partial overwrite by partition if available, or merge.
The original code used merge_delta_data in common_functions.

In [ ]:
# Ensure we use the common function if we want merge logic.
# merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
# merge_delta_data(results_final_df, 'f1_silver', 'results', processed_folder_path, merge_condition, 'race_id')

# Or simpler:
results_final_df.write.mode("overwrite").partitionBy("race_id").format(
    "delta"
).saveAsTable("f1_silver.results")

In [ ]:
dbutils.notebook.exit("Success")